### Callin Switzer
#### Make video from photos in directory, based on photos, determined by time of recording

Print date and time on top of video

Indicate frames where the bee was rewarded

Loop through all directories in a folder and make videos from images!




11 Dec 2016

19 Dec 2016 -- Updated to use opencv, insetead of matplotlib for adding text to files




In [3]:
import cv2
import numpy as np
import matplotlib
matplotlib.use("TkAgg") # have to use this for tkinter to  work below
from matplotlib import pyplot as plt
%matplotlib tk

# scikit image
import skimage
from skimage import io

import os
import pandas as pd
import time

from datetime import datetime
import shutil

/Users/callinswitzer/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [4]:
%qtconsole

In [5]:
import Tkinter as tk
import tkFileDialog

def file_choose():
    '''
    Opens tkfileDialog and returns full path name.
    It's the equivalent of file.choose in R.
    Make sure the following modeules are loaded
    import Tkinter as tk
    import tkFileDialog
    Also, won't work if %matplotlib qt4 has been used
    '''
    root = tk.Tk()
    root.update()
    file1 = tkFileDialog.askopenfilename(parent=root)
    root.destroy() # closes the window
    return file1

In [6]:
# adjust gamma
def adjust_gamma(image, gamma=1.0):
    # build a lookup table mapping the pixel values [0, 255] to
    # their adjusted gamma values
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255
        for i in np.arange(0, 256)]).astype("uint8")
 
    # apply gamma correction using the lookup table
    return cv2.LUT(image, table)

In [ ]:
#file_choose()

In [ ]:

baseDir = "/Volumes/My Passport/BeeSonicationLearningWithAvery/BeeFrequencyLearning/"
os.chdir(baseDir)

In [ ]:
beeTrialList =  [ f for f in os.listdir(".") if not f.startswith(r".") ]
beeTrialList[0:5]

In [ ]:
# start of loop

# check number 73
# start at 155

for trialNum in range(155, len(beeTrialList)):
    os.chdir(baseDir)

    CurrentTrialDir = os.path.join( os.getcwd() , beeTrialList[trialNum])
    trialFolder = [ f for f in os.listdir(CurrentTrialDir) if not f.endswith(r".txt") and not f.startswith(r".") ]

    trialMeta = [ f for f in os.listdir(CurrentTrialDir) if  f.endswith(r".txt") and not f.startswith(r".") ]
    tm1 = pd.read_csv(os.path.join(CurrentTrialDir, trialMeta[0] ), header = None)

    # replace leading spaces
    tm1.ix[:,2] = [cc.replace(" ", "") for cc in tm1.ix[:,2]]

    trialFolderDir = os.path.join(CurrentTrialDir, trialFolder[0])
    photoList = [p for p in os.listdir(trialFolderDir) if p.endswith(r".pgm")]

    # remove background photo from photolist
    photoList.remove('background.pgm')

    # sort list by date created
    creDate = [os.stat(os.path.join(trialFolderDir, c)).st_mtime for c in  photoList]
    creD_time = [datetime.fromtimestamp(aa).strftime('%Y-%m-%d %H:%M:%S') for aa in creDate]

    sortedPhotos = np.array(photoList)[np.argsort(np.array(creDate))]

    photoTimes = [c[:24] for c in sortedPhotos]



    # rewards
    rewArr = np.array(tm1.ix[:,2][tm1.ix[:,4] == " T"])

    rewd1 = [photoTimes[kk] in rewArr for kk in range(len(photoTimes))]

    # label rewards -- 
    rr = np.cumsum(rewd1)

    # make file with metadata from video
    dd = pd.DataFrame({'photoName':sortedPhotos, 'photoTime':photoTimes,
                       'vidFrame':range(1, len(sortedPhotos) + 1), 
                      'dateModified':np.array(creD_time)[np.argsort(np.array(creDate))], 
                      'rewardNum':rr, 
                      'rewardTF': rewd1})




    photoPaths = [os.path.join(trialFolderDir, c) for c in  sortedPhotos]
    # change directory
    newD = '/Users/callinswitzer/Desktop/CompressedBeeSonicationData/' + str(beeTrialList[trialNum]) + str(r"_compressed/")
    if not os.path.isdir(newD):
        os.mkdir(newD)


    # add text to images using opencv
    # about 30 times faster than using matplotlib (below)
    stt = time.time()
    for kk in range(len(sortedPhotos)):

        # plt.clf()

        img = cv2.imread(photoPaths[kk], 0)


        s1 =  sortedPhotos[kk].split(".")[0]
        try:
            s2 = s1.split("__")
            s2[0] = s2[0].replace('_', '-')
            s2[1] = s2[1][:8].replace('_', ':') + '.' + s2[1][9:12]
            s3 = s2[0] + " " + s2[1]
        except:
            s3 = "No Timestamp"

        s4 = 'REWARD ' + str(rr[kk])
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img, s3, (20, 60), font,  1.3, (0,0,0), thickness = 3) 

        if dd['rewardTF'][kk]:
            s4 = "REWARD " + str(rr[kk])
            cv2.putText(img, "REWARD " + str(rr[kk]), (20, 150), font,  1.3, (0,0,0), thickness = 3) 

        cv2.imwrite(newD + str(kk).zfill(4) + '.png', img)
#         print str(kk) + ' of ' + str(len(sortedPhotos))

#     print time.time() - stt # get time to complete

    # io.imshow(img)


    os.chdir(newD)
    # use ffmpeg to convert directory to video
    stta = time.time()
    !ffmpeg -start_number 0 -r 24 -i %04d.png -vf "scale=trunc(iw/2)*2:trunc(ih/2)*2" -c:v libx264 -pix_fmt yuv420p -y trialVid.mp4
#     print time.time() - stta # ~ 1.4 seconds

    # delete all png files in directory
    filelist = [ f for f in os.listdir(newD) if f.endswith(".png") ]

    for f in filelist:
        os.remove(f)

    # copy ampfreq to new directory
    shutil.copy2(os.path.join(CurrentTrialDir, trialMeta[0] ), newD)

    print str(trialNum) + " of " + str(len(beeTrialList)  - 1)

# END OF LOOP

In [17]:
# go through directory and rename videos to reflect the trial
cDir = '/Users/callinswitzer/Desktop/CompressedBeeSonicationData/'
os.chdir(cDir)

cFolders = [f for f in os.listdir(os.getcwd()) if not f.startswith(r".")]

In [21]:
for kk in cFolders:
    os.chdir(cDir)
    os.chdir(kk)
    os.rename('trialVid.mp4', kk + ".mp4")

In [ ]:
## add text using matplotlib
# slower than opencv

stt = time.time()
for kk in range(len(sortedPhotos)):
    img = cv2.imread(photoPaths[kk], 0)


    s1 =  sortedPhotos[kk].split(".")[0]
    try:
        s2 = s1.split("__")
        s2[0] = s2[0].replace('_', '-')
        s2[1] = s2[1][:8].replace('_', ':') + '.' + s2[1][9:12]
        s3 = s2[0] + " " + s2[1]
    except:
        s3 = "No Timestamp"

    plt.clf()
    io.imshow(img)
    plt.axis([0, img.shape[1], img.shape[0], 0])
    plt.axis('off')

    plt.gca().set_position([0,0,1,1])
    plt.gca().set_aspect('equal')
    plt.text(850, 60, s3)
    #if s2[1]
    if dd['rewardTF'][kk]:
        plt.text(850, 90, "REWARD " + str(rr[kk]))


    # save figure
    plt.savefig(newD + str(kk).zfill(4) + '.png')
    print str(kk) + ' of ' + str(len(sortedPhotos))

print time.time() - stt

